In [29]:
import json
import os
import pickle
import pandas as pd

In [53]:
tmp = {"1": 3, "2": 4}

In [58]:
{k:v for k, v in tmp.items()}

{'1': 3, '2': 4}

In [50]:
server = 'http://166.111.138.149:4000'
path = 'home/iyuge2/Project/M-SENA/Datasets'

In [51]:
os.path.join(server, path)

'http://166.111.138.149:4000/home/iyuge2/Project/M-SENA/Datasets'

In [32]:
with open(os.path.join('/home/iyuge2/Project/M-SENA/Datasets', 'config.json'), 'r') as fp:
    dataset_config = json.load(fp)

In [2]:
import pandas as pd
from tqdm import tqdm

In [36]:
df = pd.read_csv('/home/iyuge2/Project/M-SENA/Datasets/DemoDataset/label.csv', dtype={"video_id": "str", "clip_id": "str", "text": "str"})

In [43]:
df = df.drop(['label_M'], axis=1)

In [46]:
df.head(3)

,video_id,clip_id,text,label,annotation,mode,label_by
0,video_0001,0001,我不想嫁给李茶,1,Negative,train,1
1,video_0001,0002,你这是嫁入豪门啊！,3,Positive,train,0
2,video_0001,0003,我不想嫁入什么豪门，我们不就是豪门吗？,1,Negative,test,0


In [11]:
annotations = list(df['annotation'])

In [13]:
annotation_dict = {"Negative": 0, "Weakly Negative": 1, "Neutral": 2, "Positive": 3, "Weakly Positive": 4}

In [14]:
labels = [annotation_dict[k] for k in annotations]

In [47]:
df.to_csv('/home/iyuge2/Project/M-SENA/Datasets/DemoDataset/label.csv', index=None)

In [19]:
df = pd.read_csv('/home/iyuge2/Project/M-SENA/Datasets/DemoDataset/label.csv', dtype={"clip_id": str})

In [20]:
df['label_by'].value_counts()

0    1000
3     905
1     189
2     187
Name: label_by, dtype: int64

In [10]:
annotations = list(df['annotation'])

In [12]:
new_labels = [annotation_dict[v] for v in annotations]

In [14]:
df['label'] = new_labels

In [21]:
for i in tqdm(range(len(df))):
    clip_id = df.loc[i, 'clip_id']
    clip_id = str(clip_id).zfill(4)
    df.loc[i, 'clip_id'] = clip_id

100%|██████████| 2281/2281 [00:00<00:00, 4665.34it/s]


In [16]:
df.to_csv('/home/iyuge2/Project/M-SENA/Datasets/DemoDataset/label.csv', index=None, encoding='utf-8')

In [23]:
import pickle

In [41]:
with open('/home/iyuge2/Project/M-SENA/Datasets/SIMS/Processed/features/unaligned_39.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [42]:
data['train'].keys()

dict_keys(['feature_T', 'feature_A', 'feature_V', 'label_T', 'label_A', 'label_V', 'label_M', 'ids', 'raw_text', 'text_bert', 'audio_lengths', 'vision_lengths'])

five classes{[-1.0, -0.7], (-0.7, -0.1], (-0.1, 0.1], (0.1, 0.7], (0.7, 1.0]

In [43]:
data['train']['label_T'][:10]

array([1., 4., 4., 3., 0., 2., 3., 0., 0., 0.])

In [44]:
for mode in ['train', 'valid', 'test']:
    for item in ['label_T', 'label_A', 'label_V', 'label_M']:
        labels = data[mode][item]
        for i in range(len(labels)):
            if labels[i] <= 1:
                labels[i] = 0
            elif labels[i] == 2:
                labels[i] = 1
            else:
                labels[i] = 2
        data[mode][item] = labels

In [46]:
with open('/home/iyuge2/Project/M-SENA/Datasets/SIMS/Processed/features/unaligned_39.pkl', 'wb') as fp:
    pickle.dump(data, fp)

In [18]:
for mode in ['train', 'valid', 'test']:
    ids = data[mode]['ids']
    for i, cur_id in enumerate(ids):
        ids[i] = cur_id[:-5] + '-' + cur_id[-4:]
    data[mode]['ids'] = ids

In [19]:
with open('/home/iyuge2/Project/M-SENA/Datasets/SIMS/Processed/features/unaligned_39.pkl', 'wb') as fp:
    pickle.dump(data, fp)

In [37]:
new_data = {}
for mode in ['train', 'valid', 'test']:
        cur_ids = data[mode]['ids']
        for i in range(len(cur_ids)):
            ids = cur_ids[i].split('_')
            video_id = '_'.join(ids[:-1])
            clip_id = ids[-1]
            new_k = video_id + '-' + clip_id
            tmp = {}
            for k in ['feature_T', 'feature_A', 'feature_V', 'label_M', 'raw_text', 'text_bert', 'audio_lengths', 'vision_lengths']:
                tmp[k] = data[mode][k][i]
            new_data[new_k] = tmp

In [38]:
with open('/home/iyuge2/Project/M-SENA/Datasets/DemoDataset/features/unaligned_39.pkl', 'wb') as fp:
    pickle.dump(new_data, fp)

In [59]:
df = pd.read_csv('/home/iyuge2/Project/M-SENA/Datasets/SIMS/label.csv', dtype={"video_id": "str", "clip_id": "str", "text": "str"})

In [66]:
annotation_dict = {
                "Negative": 0,
                "Neutral": 1,
                "Positive": 2
            }
tmp_dict = {
                "Weakly Negative": "Negative",
                "Weakly Positive": "Positive",
                "Positive": "Positive",
                "Neutral": "Neutral",
                "Negative": "Negative"
            }

In [67]:
annotations = list(df['annotation'])
new_annotations = [tmp_dict[v] for v in annotations]

In [69]:
new_labels = [annotation_dict[v] for v in new_annotations]

In [70]:
df['label'] = new_labels
df['annotation'] = new_annotations

In [71]:
df.head(10)

,video_id,clip_id,text,label,annotation,mode,label_by
0,video_0001,0001,我不想嫁给李茶,0,Negative,train,0
1,video_0001,0002,你这是嫁入豪门啊！,2,Positive,train,0
2,video_0001,0003,我不想嫁入什么豪门，我们不就是豪门吗？,0,Negative,test,0
3,video_0001,0004,有那么明显吗？,0,Negative,train,0
4,video_0001,0005,我在这消费这么多钱，我低血糖吃块糖还不行了,0,Negative,train,0
5,video_0001,0006,够了，每次都是这一套，我不想再听了。,0,Negative,test,0
6,video_0001,0007,我刚才信号不好，现在可以了，怎么样，妈，公司给我安排的海景别墅楼。,2,Positive,train,0
7,video_0001,0008,我刚才是一害怕我就跟王安迪撒了个谎,0,Negative,train,0
8,video_0001,0009,说到演讲啊，我不禁想起了我在我的母校。,2,Positive,test,0
9,video_0001,0010,经理经理，别忘了我那五十万和副科长。,2,Positive,train,0


In [37]:
annotations = list(df['annotation'])
labels = [annotation_dict[v] for v in annotations]

In [39]:
df['label'] = labels

In [72]:
df.to_csv('/home/iyuge2/Project/M-SENA/Datasets/SIMS/label.csv', index=None)

In [73]:
import time

In [74]:
t = time.time()

In [80]:
print(str(round(t * 1000)))

1609853739306


In [81]:
import pickle

In [82]:
with open('/home/iyuge2/Project/M-SENA/M-SENA-Backend/M-Codes/results/LF_DNN-SIMS-12.pkl', 'rb') as fp:
    features = pickle.load(fp)

In [84]:
features.keys()

dict_keys(['train', 'valid', 'test'])

In [86]:
features['train'].keys()

dict_keys(['Feature_T', 'Feature_A', 'Feature_V', 'Feature_M'])

In [89]:
import numpy as np
a = np.ones([3,5])

In [91]:
a.tolist()

[[1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 1.0, 1.0]]

In [97]:
a = np.array([i for i in range(18)])

In [103]:
ai = np.where(a > 4)[0].tolist()

In [104]:
a[ai]

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [96]:
a[[1,2]]

array([[ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17]])